In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

from modules.mail import send_checks_mail

import sys
from config.variables import user

In [2]:
training_code = 'SEC'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude 
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
4445,00000000,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2019-08-17 00:00:00
4985,00000019,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-10-24 00:00:00
4747,00000020,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-05-07 00:00:00
4618,00000021,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-26 00:00:00
4265,00000022,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2020-09-18 12:02:21
...,...,...,...,...,...
2410,00042079,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2020-09-22 16:47:56
2084,00042099,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2020-09-22 16:49:13
4033,00042153,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"9,00",2020-09-22 16:51:26
2872,00044847,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Reprovado,"0,00",2020-02-14 00:00:00


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
98905,00002976,INSS
98923,00034570,INSS
98925,00035529,INSS
98930,00013481,INSS
98935,00021086,INSS
...,...,...
100249,00032174,INSS
100251,00035516,INSS
100260,00022936,INSS
100268,00024216,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
98901,00001589,LIC-SREM
98904,00002846,LIC-SREM
98921,00032713,LIC-SREM
98924,00035305,LIC-SREM
98946,00011808,LIC-SREM
...,...,...
100245,00046272,LIC-SREM
100246,00046278,LIC-SREM
100247,00046284,LIC-SREM
100248,00046306,LIC-SREM


In [9]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep', 'arr']]
rosters['duration'] = pd.to_datetime(rosters['arr']) - pd.to_datetime(rosters['dep'])
rosters['duration'] = rosters['duration'].astype('timedelta64[h]').astype(int)
rosters = rosters[rosters['duration'] < 6.0]
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

,cif,code,fr,dep
83976,00000021,C-SEC-ON,CGH,2021-01-13 16:30:00
97309,00000022,C-ENS-SEC,CGH,2020-09-18 11:30:00
41643,00000024,C-ENS-SEC,CGH,2020-09-08 16:30:00
142233,00000026,C-ENS-SEC,CGH,2020-11-24 11:30:00
41044,00000027,C-SEC-ON,CGH,2020-12-07 16:30:00
...,...,...,...,...
112203,00042333,C-SEC-ON,GIG,2020-12-18 16:32:00
132383,00042334,C-SEC-ON,FOR,2020-12-21 16:32:00
74765,00042351,C-ENS-SEC,GIG,2020-11-13 11:30:00
86144,00042386,C-SEC-ON,FOR,2020-12-14 16:31:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
63,00046225,SEC,2022-09-30
73,00034571,SEC,2022-10-31
74,00002976,SEC,2020-11-27
78,00024543,SEC,2022-12-31
81,00024238,SEC,2022-12-31
...,...,...,...
54870,00005686,SEC,2021-11-30
54871,00022337,SEC,2021-11-30
54982,00038070,SEC,2022-01-31
54983,00042289,SEC,2022-01-31


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,SEC,2022-11-30,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-05-07 00:00:00,NaN,NaN,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-26 00:00:00,C-SEC-ON,2021-01-13 16:30:00,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,SEC,2022-09-30,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2020-09-18 12:02:21,C-ENS-SEC,2020-09-18 11:30:00,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,SEC,2022-09-30,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2020-09-08 16:22:29,C-ENS-SEC,2020-09-08 16:30:00,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,SEC,2022-11-30,NaN,NaN,NaN,NaT,C-ENS-SEC,2020-11-24 11:30:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
1,00000021,GRABLER,CMT,IBX,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-26,C-SEC-ON,2021-01-13,NaN,NaN,843,False
130,00001161,FERNANDO SIDNEI,CMT,IR,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-06,C-SEC-ON,2021-01-13,NaN,NaN,863,False
168,00001675,ROGERIO MACHADO,CMT,IRX,GIG,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-26,C-SEC-ON,2021-01-13,NaN,NaN,843,False
291,00002836,GOES,CMT,IR,BSB,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"8,00",2018-09-11,C-SEC-ON,2021-01-13,NaN,NaN,858,False
344,00003096,LEONARDO CONSTANT,CMT,IR,CGH,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"9,00",2018-09-26,C-SEC-ON,2021-01-13,NaN,NaN,843,False
378,00003368,PONZONI,CMT,IR,CGH,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-06,C-SEC-ON,2021-01-13,NaN,NaN,863,False
554,00005532,LOPES,CMT,NaN,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-06,C-SEC-ON,2021-01-13,NaN,NaN,863,False
674,00008225,COUTINHO,CMT,IR,GIG,SEC,2020-12-29,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-08-28,C-SEC-ON,2020-12-09,NaN,LIC-SREM,872,False
802,00009770,PASQUA,CMT,NaN,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-06,C-SEC-ON,2021-01-13,NaN,NaN,863,False
808,00009826,BARANHUK,CMT,NaN,GRU,SEC,2021-01-28,AVSEC PARA TRIPULANTES - ATUALIZAÇÃO RBAC 110,Concluído,"10,00",2018-09-06,C-SEC-ON,2021-01-13,NaN,NaN,863,False


In [13]:
validation.to_csv(f'./data/output/{training_code}.csv', index=False, sep=';', encoding='utf-8-sig')

In [14]:
# Arquivo de import
# fill table
df_atualizar = validation[validation['atualizar']]

df_atualizar

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar


In [15]:
df_import = df[['cif', 'checkTypeCode', 'data_conclusao']]

# fill table

data = df_import.copy()

data['update'] = 'U'
data['id'] = df_['cif'].astype(str).str.zfill(8)
data['type'] = 'SEC'
data['issue'] = df_['data_conclusao'].dt.strftime('%Y%m%d') + '0000'
data['expiry'] = ((df_['data_conclusao'] + \
                   pd.offsets.MonthEnd(0)) + \
                   pd.offsets.DateOffset(years=1)).dt.strftime('%Y%m%d') + '0000'

data['extended'] = data['expiry']
data['failed'] = 'N'
data['do_not_plan_before'] = data['expiry']
data['zero'] = '0'
data['remark'] = ''
data['check_comment'] = ''
data['confirmed'] = 'Y'
data['paper_recieved'] = 'Y'

# remove fields from original file
cols = [
    'update',
    'cif',
    'type',
    'issue',
    'expiry',
    'extended',
    'failed',
    'do_not_plan_before',
    'zero',
    'remark',
    'check_comment',
    'confirmed',
    'paper_recieved'  
]

data = data[cols].copy()

data.to_csv('./data/output/crm_import_sec.dat', sep='|', header=None, index=False, line_terminator='\r\n')

data

,update,cif,type,issue,expiry,extended,failed,do_not_plan_before,zero,remark,check_comment,confirmed,paper_recieved


In [16]:
out_email = validation.copy()

out_email = out_email[~out_email['inss'].isin(['INSS'])]
out_email = out_email[~out_email['lnr'].isin(['LIC-SREM'])]

out_email = out_email[['cif',
                       'guerra',
                       'funcao',
                       'base',
                       'checkTypeCode',
                       'expiryDt', 
                       'status', 
                       'nota', 
                       'data_conclusao',
                       'code',
                       'dep',
                       'tempo_de_conclusao (dias)',
                       'atualizar']]

out_email['expiryDt'] = out_email['expiryDt'].dt.strftime('%d/%m/%Y')
out_email['data_conclusao'] = out_email['data_conclusao'].dt.strftime('%d/%m/%Y')
out_email['dep'] = out_email['dep'].dt.strftime('%d/%m/%Y')
out_email['status'] = out_email['status'].fillna('')
out_email['nota'] = out_email['nota'].fillna('')

In [17]:
send_checks_mail(out_email, user['mail'], user['password'], user['emails'])